<a href="https://colab.research.google.com/github/naveen777-github/Machine-Generated-Text-Detection-Using-LSTMS/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**LSTM-based RNN**

### Get data for train/dev/test sets

In [ ]:
!pip install datasets
!gdown --folder https://drive.google.com/drive/folders/1vESAUKz1IG6L6vo-zgGuBHBFYxeJev97?usp=sharing --fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Retrieving folder contents
Processing file 1NU65pQ_ClBuJFnXm_7h73mYjcmH7Y86M en_dev.jsonl
Processing file 1sxbZ7fAk27ri9n8uQWIz_lJN4Oron1pp en_devtest.jsonl
Pro

In [ ]:
import pandas as pd
import os
import logging

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
random_seed = 0
folder_path = 'data/'
train_path = folder_path + 'en_train.jsonl'
dev_path = folder_path + 'en_dev.jsonl'
test_path = folder_path + 'en_devtest.jsonl'
prediction_path = folder_path + 'en_prediction.jsonl'


if not os.path.exists(train_path):
    logging.error("File doesnt exists: {}".format(train_path))
    raise ValueError("File doesnt exists: {}".format(train_path))

if not os.path.exists(dev_path):
    logging.error("File doesnt exists: {}".format(dev_path))
    raise ValueError("File doesnt exists: {}".format(dev_path))

if not os.path.exists(test_path):
    logging.error("File doesnt exists: {}".format(test_path))
    raise ValueError("File doesnt exists: {}".format(test_path))


id2label = {0: "human", 1: "machine"}
label2id = {"human": 0, "machine": 1}

In [ ]:
def get_data(train_path, dev_path, test_path, random_seed):
    """
    function to read dataframe with columns
    """
    # train_df = pd.read_json(train_path, lines=True)
    train_df = pd.read_json(train_path, lines=True).head( 6160767)
    val_df = pd.read_json(dev_path, lines=True).head(400)
    test_df = pd.read_json(test_path, lines=True)

    return train_df, val_df, test_df

In [ ]:
train_df, valid_df, test_df = get_data(train_path, dev_path, test_path, random_seed)

### Load pretrained word2vec

In [ ]:
!gdown https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing --fuzzy
!gunzip GoogleNews-vectors-negative300.bin.gz

Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

but Gdown can't. Please check connections and permissions.
gzip: GoogleNews-vectors-negative300.bin.gz: No such file or directory


In [ ]:
from gensim.models import KeyedVectors

word2vec_file = 'GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(word2vec_file, binary=True)

pretrained_embeddings = word2vec

FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'

In [ ]:
word_vector = word2vec['apple']
print(word_vector)
print(word_vector.shape[0])

NameError: name 'word2vec' is not defined

### Preprocessing to reduce the vocabulary

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import string
import re
from nltk.corpus import stopwords

def preprocess_data(data):
  stop_words = set(stopwords.words('english'))

  text = ' '.join(data) #join all text in one single string
  text = text.lower() #make everything lower case
  text = text.replace('\n', ' ') #remove \n characters
  text = re.sub(r'[^\w\s]', ' ', text) #remove any punctuation or special characters

  # text = re.sub(r'_', ' ', text) # remove underline character
  text = re.sub(r'\d', ' ', text) #remove all numbers
  text = ' '.join(word for word in text.split() if word not in stop_words) #remove all stopwords (see imports to help you with this)

  return text

In [ ]:
text=preprocess_data(train_df['text'])

In [ ]:
pad_identifier = '<PAD>'
oov_identifier = '<UNK>'

def vocab_frequency(text):
  vocab_dict = {}
  vocab_dict[pad_identifier] = 0
  words = text.split()
  for word in words:
    vocab_dict[word] = vocab_dict.get(word, 0) + 1
  vocab_dict[oov_identifier] = 0
  return vocab_dict

In [ ]:
vocabulary = vocab_frequency(text)
print(len(vocabulary))
print(vocabulary[pad_identifier], vocabulary[oov_identifier]) # check if vocabulary contains 'oov'

528531
0 0


### Check how many of the vocabulary are not in pretrained_embeddings

In [ ]:
oov_count = -1

for word in vocabulary.keys():
    if word not in word2vec:
        oov_count += 1

print(oov_count)
print(word2vec['<unk>']) # check if pretrained word2vec contains 'oov'

NameError: name 'word2vec' is not defined

### Format dataset to meet the needs of LSTM

In [ ]:
def create_word_to_index(vocabulary):
  word_to_index = {}
  index = 0
  for word in vocabulary:
    word_to_index[word] = index
    index += 1
  return word_to_index

In [ ]:
word_to_index = create_word_to_index(vocabulary)
print(len(word_to_index))
print(word_to_index[pad_identifier], word_to_index[oov_identifier])

index_to_word = {}
for vocab_word, index in word_to_index.items():
    index_to_word[index] = vocab_word
print(index_to_word[0])

528531
0 528530
<PAD>


In [ ]:
def text_to_sequence(text, word_to_index):
    sequence=[]
    for word in text.split():
         sequence.append(word_to_index.get(word, word_to_index[oov_identifier]))
    return sequence


def prepare_dataset(dataset, word_to_index):
    ids = dataset['id']
    sequences = [text_to_sequence(preprocess_data(row['text']), word_to_index) for _, row in dataset.iterrows()]
    labels = dataset['label']
    assert len(sequences) == len(labels)
    return ids, sequences, labels # cannot be converted to tensor before padding each sequence to same length within a batch

In [ ]:
def sequences_statistics(sequences):
  # compute average length of sequences after preprocessing & print the smallest & largest lengths of sequence
  total_length = sum(len(seq) for seq in sequences)
  average_length = total_length / len(sequences)
  min_length = min(len(seq) for seq in sequences)
  max_length = max(len(seq) for seq in sequences)
  print(f"Average sequence length: {average_length}")
  print(f"Smallest sequence length: {min_length}")
  print(f"Largest sequence length: {max_length}")

In [ ]:
ids, sequences, labels = prepare_dataset(train_df, word_to_index)
print(len(sequences))
print(train_df['text'][0])

sequences_statistics(sequences)

610767
Hitler's plans for the succession and power structure after his death are shrouded in mystery, as he never explicitly wrote down his intentions. However, it is known that he designated several potential successors, including Heinrich Himmler, Hermann Göring, and Joseph Goebbels, each with their own strengths and weaknesses. 

In the final days of his life, Hitler grew increasingly paranoid and isolated, leading to rumors of a civil war within the Nazi Party. He became convinced that the Party was plotting against him, and his trusted advisors, including Goering and Himmler, were secretly vying for power. 

To counter this perceived threat, Hitler reportedly planned to appoint a successor, most likely Himmler, in a 'Fuhrer-proof' mechanism. This would have granted Himmler absolute power over the Party and state, rendering the concept of democratic succession irrelevant. Hitler also envisioned a hierarchical, hereditary power structure, with his loyal followers and their families 

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

MAX_SEQ_LENGTH = 512

def collate_fn(batch):
    ids, sequences, labels = zip(*batch)
    sequences = [torch.tensor(seq[:MAX_SEQ_LENGTH]) for seq in sequences] # Truncating to the first MAX_SEQ_LENGTH token
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=word_to_index[pad_identifier]) # Pad sequences
    labels = torch.tensor(labels) # Convert labels to tensors
    return ids, sequences_padded, labels

In [ ]:
def dataloader_statistics(dataloader):
  num_batches = len(dataloader)
  print(f"Number of batches: {num_batches}")
  for batch in dataloader:
      print(len(batch))
      ids, sequences_padded, labels = batch
      print(ids)
      print(sequences_padded.shape)  # Shape of the padded sequences
      print(labels.shape)
      print(sequences_padded)
      print(labels)
      break

In [ ]:
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn


torch.manual_seed(random_seed)
train_dataloader = DataLoader(list(zip(ids, sequences, labels)), batch_size=64, shuffle=True, collate_fn=collate_fn)

dataloader_statistics(train_dataloader)

Number of batches: 9544
3
('e5e21281-ff78-45e2-8fd1-a4886969940f', 'fae80b93-e920-44ba-a180-9401b9b23487', '4c81361b-a5f0-40ae-b881-51cf9a467214', '80b40430-a79c-4507-aed0-b630c2376c94', '80a75b1f-d482-4787-ad7d-d46cb9dc0e98', 'd60382ad-ab64-4935-8a62-2dd1635f1aae', 'ff0f7d2b-b220-4f0c-99e4-72da48fa5be0', '16fcd893-8106-4160-a968-36943c877335', '0fdd97c3-d55a-43df-ae65-e88a1a3fc171', '407a1572-496f-4148-a1f4-88e044170776', '56875331-bc46-459b-ad7f-73fc73492a28', '04bb8208-d35d-4cae-b8ec-7fbf1fe779a5', '1205d062-db40-4c4e-b269-be623d522b75', '48c9d131-5d59-4b1f-821a-31efd57d3897', 'ec5c9a25-0c50-4121-b886-36ce4aad905f', '7e8e3a87-7f63-4243-8ad1-4112a6f60a27', 'c923575f-2af2-4df1-ae13-b3c407b7a4e8', '538a7441-8b3f-48c2-b33d-086dad1fc1a9', '7fab60ea-a06d-4470-b40e-7badbfc3f082', 'c3c6f2cc-7dfd-49f6-89a9-95a832bb55e3', 'b62bfc87-9cee-49a6-acf5-a190e3b7de54', '68fadc2a-2ef7-4032-8922-7245e0fe9aff', '8880e366-37bd-4b7c-96f3-33ee4e578a1d', 'ce2695bb-db31-4e13-91a0-ddc53104b287', 'f51bf051-a9c

### Define the model

In [ ]:
class RNN_FFNN_Sigmoid(nn.Module):
    def __init__(self, vocab_size, embed_dim=300, hidden_size=128, output_size=1, num_layers=2, dropout=0.5, ff_hidden_size=64):
        super(RNN_FFNN_Sigmoid, self).__init__()

        # TODO:
        # For words not in pretrained_embeddings, random initialization is used,
        # while pre-trained embeddings are used for words that are present in pretrained_embeddings

        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=word_to_index[pad_identifier])
        self.rnn = nn.LSTM(embed_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)

        # FFNN layers
        self.fc1 = nn.Linear(hidden_size, ff_hidden_size)  # Hidden size to FFNN hidden size
        self.fc2 = nn.Linear(ff_hidden_size, output_size)  # FFNN hidden size to output size

    def forward(self, x):
        embeddings = self.embedding(x)  # Embedding layer
        _, (hidden, _) = self.rnn(embeddings)  # RNN layer
        last_hidden = hidden[-1]  # Get the last hidden state

        # Pass through FFNN layers
        ff_output = F.relu(self.fc1(last_hidden))  # First FC layer with ReLU
        output = self.fc2(ff_output)  # Second FC layer

        return torch.sigmoid(output)  # Apply sigmoid for classification

In [ ]:
class BiLSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=300, hidden_size=128, output_size=1, num_layers=2, dropout=0.5, ff_hidden_size=64):
        super(BiLSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=word_to_index[pad_identifier])
        self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.fc1 = nn.Linear(hidden_size * 2, ff_hidden_size)
        self.fc2 = nn.Linear(ff_hidden_size, output_size)

    def forward(self, x):
        embeddings = self.embedding(x)
        _, (hidden, _) = self.lstm(embeddings)

         # Concatenate the forward and backward hidden states from the last LSTM layer
        last_hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        ff_output = F.relu(self.fc1(last_hidden))
        output = self.fc2(ff_output)

        return torch.sigmoid(output)

### Initialize & Train

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

criterion = nn.BCELoss()
epochs = 10

Using cuda


In [ ]:
torch.manual_seed(random_seed)
model = RNN_FFNN_Sigmoid(len(word_to_index)).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

In [ ]:
from tqdm import tqdm

def train_model(model, optimizer, epochs, train_dataloader):
    for epoch in range(epochs):
        epoch_loss = 0
        torch.manual_seed(random_seed)
        for _, inputs, labels in tqdm(train_dataloader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()  # Squeeze output to match labels shape
            loss = criterion(outputs, labels.float())
            loss.backward()

            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            epoch_loss += loss.item()

        # TODO: Scheduler step
        # scheduler.step(epoch_loss)
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(train_dataloader)}")

In [ ]:
train_model(model, optimizer, epochs, train_dataloader)

100%|██████████| 9544/9544 [04:13<00:00, 37.58it/s]


Epoch 1, Loss: 0.6148881981241433


100%|██████████| 9544/9544 [04:12<00:00, 37.74it/s]


Epoch 2, Loss: 0.5595670328705828


100%|██████████| 9544/9544 [04:12<00:00, 37.79it/s]


...

Epoch 8, Loss: 0.44557045079136937


100%|██████████| 9544/9544 [04:12<00:00, 37.75it/s]


Epoch 9, Loss: 0.4332641173210323


100%|██████████| 9544/9544 [04:12<00:00, 37.82it/s]

Epoch 10, Loss: 0.4209631012295188


In [ ]:
torch.manual_seed(random_seed)
model_bilstm = BiLSTMClassifier(len(word_to_index)).to(device)
optimizer_bilstm = optim.Adam(model_bilstm.parameters(), lr=0.001)
train_model(model_bilstm, optimizer_bilstm, epochs, train_dataloader)

100%|██████████| 9544/9544 [05:30<00:00, 28.87it/s]


Epoch 1, Loss: 0.5889779534987162


100%|██████████| 9544/9544 [05:30<00:00, 28.90it/s]


Epoch 2, Loss: 0.5194587409465081


100%|██████████| 9544/9544 [05:30<00:00, 28.87it/s]


...

Epoch 8, Loss: 0.3443356800249841


100%|██████████| 9544/9544 [05:30<00:00, 28.86it/s]


Epoch 9, Loss: 0.3244182093432743


100%|██████████| 9544/9544 [05:30<00:00, 28.86it/s]

Epoch 10, Loss: 0.3076476480121734


### Predict for test set

In [ ]:
def test(test_df, model, device):
    """ Function to write the prediction file for evaluation, write two columns: id, label
    """
    test_ids, test_sequences, test_labels = prepare_dataset(test_df, word_to_index)

    # print(len(test_sequences))
    # print(test_df['text'][0])
    # sequences_statistics(test_sequences)

    torch.manual_seed(random_seed)
    test_dataloader = DataLoader(list(zip(test_ids, test_sequences, test_labels)), batch_size=64, shuffle=False, collate_fn=collate_fn)

    # dataloader_statistics(test_dataloader)

    preds = []
    model.eval()
    with torch.no_grad():
        for ids, inputs, labels in test_dataloader:
            # print(ids)
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze()  # Ensure outputs are 1D
            predicted = (outputs > 0.5).int()  # Threshold at 0.5
            preds.extend(predicted.cpu().numpy()) # Convert tensors to numpy arrays

    return preds

In [ ]:
# !!!!! predict using unilstm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

preds = test(test_df, model, device)
print(type(preds))
print(preds)

Using cuda
<class 'list'>
[0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1

In [ ]:
# !!!!! predict using bilstm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

preds = test(test_df, model_bilstm, device)
print(type(preds))
print(preds)

Using cuda
<class 'list'>
[0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1

In [ ]:
predictions_df = pd.DataFrame({'id': test_df['id'], 'label': preds})
predictions_df.to_json(prediction_path, lines=True, orient='records')

### Evaluate

In [ ]:
import os
import argparse
import logging
import json
import pandas as pd


logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
COLUMNS = ['id', 'label']


def check_format(file_path):
  if not os.path.exists(file_path):
    logging.error("File doesnt exists: {}".format(file_path))
    return False

  try:
    submission = pd.read_json(file_path, lines=True)[['id', 'label']]
  except:
    logging.error("File is not a valid json file: {}".format(file_path))
    return False

  for column in COLUMNS:
    if submission[column].isna().any():
      logging.error("NA value in file {} in column {}".format(file_path, column))
      return False

  if not submission['label'].isin(range(0, 2)).all():
    logging.error("Unknown Label in file {}".format(file_path))
    logging.error("Unique Labels in the file are {}".format(submission['label'].unique()))
    return False

  return True

In [ ]:
import logging.handlers
import argparse
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd
import sys
sys.path.append('.')


def evaluate(pred_fpath, gold_fpath):
  """
    Evaluates the predicted classes w.r.t. a gold file.
    Metrics are: f1-macro, f1-micro and accuracy

    :param pred_fpath: a json file with predictions,
    :param gold_fpath: the original annotated gold file.

    The submission of the result file should be in jsonl format.
    It should be a lines of objects:
    {
      id     -> identifier of the test sample,
      labels -> labels (0 or 1 for subtask A and from 0 to 5 for subtask B),
    }
  """

  pred_labels = pd.read_json(pred_fpath, lines=True)[['id', 'label']]
  gold_labels = pd.read_json(gold_fpath, lines=True)[['id', 'label']]
  # gold_labels = pd.read_json(gold_fpath, lines=True)[['id', 'label']].head(400)

  merged_df = pred_labels.merge(gold_labels, on='id', suffixes=('_pred', '_gold'))

  macro_f1 = f1_score(merged_df['label_gold'], merged_df['label_pred'], average="macro", zero_division=0)
  micro_f1 = f1_score(merged_df['label_gold'], merged_df['label_pred'], average="micro", zero_division=0)
  accuracy = accuracy_score(merged_df['label_gold'], merged_df['label_pred'])

  return macro_f1, micro_f1, accuracy


def validate_files(pred_files):
  if not check_format(pred_files):
    logging.error('Bad format for pred file {}. Cannot score.'.format(pred_files))
    return False
  return True

In [ ]:
pred_file_path = prediction_path
gold_file_path = test_path

with open(pred_file_path, 'r', encoding='utf-8') as pred_file:
    pred_first_line = pred_file.readline()
    # pred_first_line = pred_file.read()
    print(f"Prediction file first line:\n{pred_first_line}")

with open(gold_file_path, 'r', encoding='utf-8') as gold_file:
    gold_first_line = gold_file.readline()
    print(f"Gold file first line:\n{gold_first_line}")

Prediction file first line:
{"id":"67688297-742f-4e57-83d9-d88b9778578c","label":0}

Gold file first line:
{"id": "67688297-742f-4e57-83d9-d88b9778578c", "source": "raid", "model": "human", "label": 0, "text": "This past month has been god awful financial wise. My partner lost his job so I have had to support us. Then his car got completely wrecked and public transportation in the city we live in is unreliable, so even if he wanted to get a job he wouldn\u2019t be able to get around. There\u2019s no available work from home jobs either and I check daily. It costs a lot to fix the car, and it also costs a lot to buy a car. For the past two weeks we have been so broke and poor we couldn\u2019t even eat. I finally got paid yesterday and we paid bills and got food, but I\u2019m back to square one of having no money left over. I\u2019m trying to study for the MCAT, but all this stress is affecting my ability to focus. I\u2019m breaking out so badly from the stress. I\u2019ve had my period m

In [ ]:
if validate_files(pred_file_path):
  print('Prediction file format is correct')
  macro_f1, micro_f1, accuracy = evaluate(pred_file_path, gold_file_path)
  print("macro-F1={:.5f}\tmicro-F1={:.5f}\taccuracy={:.5f}".format(macro_f1, micro_f1, accuracy))

Prediction file format is correct
macro-F1=0.67718	micro-F1=0.70114	accuracy=0.70114
